Use https://multimeric.github.io/DelverLensExtract/ and the latest apk+export the Delver Lens file.

Then import to here.

Upload Generated cards.csv

In [ ]:
from google.colab import files
import pandas as pd
import os
import zipfile
import requests
import time
from tqdm import tqdm, trange

In [ ]:
uploaded = files.upload()
for filenames in uploaded.keys():
    print(f'Uploaded file: {filenames}')

Saving cards.csv to cards.csv
Uploaded file: cards.csv


# Run code

In [ ]:
df = pd.read_csv(filenames)
selected_columns = df[['list', 'name']]
selected_columns.to_csv('selected_cards.csv', index=False)

print("Selected columns saved to 'selected_cards.csv'.")

Selected columns saved to 'selected_cards.csv'.


In [ ]:
df = pd.read_csv('selected_cards.csv')

# Function to query Scryfall API and extract type_line and color_identity
def get_card_info(card_name):
    url = f'https://api.scryfall.com/cards/named?fuzzy={card_name}'
    response = requests.get(url)
    if response.status_code == 200:
        card_data = response.json()
        type_line = card_data.get('type_line')
        colour_identity = card_data.get('color_identity')
        return type_line, colour_identity
    else:
        print(f"Failed to retrieve data for {card_name}. Status code: {response.status_code}")
        return None, None

In [ ]:
df['Type'] = ""
df['Colour'] = ""

start_time = time.time()
ips = 0

colour_map = {
    "B": "Black",
    "G": "Green",
    "R": "Red",
    "U": "Blue",
    "W": "White"
}

# Loop over each row in the DataFrame and query Scryfall API for type_line and color_identity
for index, row in zip(trange(len(df), desc='Processing'), df.itertuples()):
    card_name = row.name
    type_line, colour_identity = get_card_info(card_name)
    if type_line is not None:
        df.at[index, 'Type'] = type_line
    if colour_identity is not None:
        if len(colour_identity) > 0:
            colours = [colour_map.get(colour, "Unknown") for colour in colour_identity]
            df.at[index, 'Colour'] = ', '.join(colours)
        else:
            df.at[index, 'Colour'] = "Colourless"

    # Calculate IPS
    elapsed_time = time.time() - start_time
    ips = 1 / elapsed_time
    start_time = time.time()
    if elapsed_time > 0.1:
        continue
    else:
        time.sleep(0.1 - elapsed_time)

df.to_csv('selected_cards_with_type.csv', index=False)

print("Type information added to 'selected_cards_with_type.csv'.")

Processing: 100%|██████████| 1895/1895 [05:56<00:00,  5.31it/s]

Type information added to 'selected_cards_with_type.csv'.


In [ ]:
df = pd.read_csv('selected_cards_with_type.csv')
unique_lists = df['list'].unique()

first_entry_names = []
num_entries = []

# Iterate through unique lists and create separate CSV files
for list_value in unique_lists:
    list_df = df[df['list'] == list_value]
    first_entry_name = list_df.iloc[0]['name']
    first_entry_names.append(first_entry_name)
    num_entries.append(len(list_df))

print("First entry names and number of entries for each list:")
for list_value, name, num in zip(unique_lists, first_entry_names, num_entries):
    print(f"List {list_value}: {name} ({num} entries)")

df['Subtype'] = ""

# Iterate over each row in the DataFrame to split 'Type' into 'Type' and 'Subtype'
for index, row in df.iterrows():
    type_line = row['Type']
    if ' — ' in type_line:
        type_, subtype = type_line.split(' — ', 1)
        df.at[index, 'Type'] = type_.strip()
        df.at[index, 'Subtype'] = subtype.strip()

df_sorted = df.sort_values(by=['Type', 'Colour', 'Subtype', 'name'])
df_sorted.to_csv('selected_cards_sorted.csv', index=False)

# Iterate through unique lists and create separate CSV files
for list_value in unique_lists:
    list_df = df[df['list'] == list_value]
    filename = f'List{list_value}_cards.csv'
    list_df.to_csv(filename, index=False)

print("DataFrame sorted and saved to 'selected_cards_sorted.csv'.")

First entry names and number of entries for each list:
List 1: Henzie Toolbox" Torre" (100 entries)
List 2: Venser, Corpse Puppet (100 entries)
List 3: Jolene, the Plunder Queen (100 entries)
List 10: Gadrak, the Crown-Scourge (162 entries)
List 11: Zedruu the Greathearted (100 entries)
List 9: The Blackstaff of Waterdeep (1333 entries)
DataFrame sorted and saved to 'selected_cards_sorted.csv'.


# Set list IDs to name

Set list id to name by checking first entry in every List

In [ ]:
from IPython.display import display, HTML
df = pd.read_csv('selected_cards_sorted.csv')

list_id_to_name = {
    1: "Henzie Toolbox Torre",
    2: "Venser, Corpse Puppet",
    3: "Jolene, the Plunder Queen",
    9: "Cards in Boxes",
    10: "Trade Binder",
    11: "Zedruu, the Greathearted"
}

zip_filename = "card_files.zip"
with zipfile.ZipFile(zip_filename, "w") as zipf:
    all_lists = []
    for list_value in unique_lists:
        list_df = df[df['list'] == list_value]
        all_lists.append(list_df)

    all_lists_df = pd.concat(all_lists)
    all_lists_filename = 'all_lists_cards.csv'
    all_lists_df.to_csv(all_lists_filename, index=False)
    zipf.write(all_lists_filename)
    os.remove(all_lists_filename)
    print(f"Created CSV file containing all lists: {all_lists_filename}")

    # Convert the DataFrame containing all lists to a .txt file with cards separated by newlines
    all_cards_txt = all_lists_df['name'].str.cat(sep='\n')
    all_cards_txt_filename = 'all_lists_cards.txt'
    with open(all_cards_txt_filename, 'w') as txt_file:
        txt_file.write(all_cards_txt)

    zipf.write(all_cards_txt_filename)
    os.remove(all_cards_txt_filename)
    print(f"Created TXT file containing all lists: {all_cards_txt_filename}")

    # Iterate through the mapping and process the CSV files
    for list_id, name in list_id_to_name.items():
        old_filename = f'List{list_id}_cards.csv'
        new_csv_filename = f'{name}_cards.csv'
        new_txt_filename = f'{name}_cards.txt'

        # Check if the old filename exists
        if os.path.exists(old_filename):
            df = pd.read_csv(old_filename)
            df = df.drop(columns=['list'])
            df.to_csv(new_csv_filename, index=False)
            cards_txt = df['name'].str.cat(sep='\n')
            with open(new_txt_filename, 'w') as txt_file:
                txt_file.write(cards_txt)

            print(f"Removed 'list' column, renamed '{old_filename}' to '{new_csv_filename}', and created '{new_txt_filename}'")
            zipf.write(new_csv_filename)
            zipf.write(new_txt_filename)

            os.remove(new_csv_filename)
            os.remove(new_txt_filename)
        else:
            print(f"File '{old_filename}' not found.")

files.download(zip_filename)

Created CSV file containing all lists: all_lists_cards.csv
Created TXT file containing all lists: all_lists_cards.txt
Removed 'list' column, renamed 'List1_cards.csv' to 'Henzie Toolbox Torre_cards.csv', and created 'Henzie Toolbox Torre_cards.txt'
Removed 'list' column, renamed 'List2_cards.csv' to 'Venser, Corpse Puppet_cards.csv', and created 'Venser, Corpse Puppet_cards.txt'
Removed 'list' column, renamed 'List3_cards.csv' to 'Jolene, the Plunder Queen_cards.csv', and created 'Jolene, the Plunder Queen_cards.txt'
Removed 'list' column, renamed 'List9_cards.csv' to 'Cards in Boxes_cards.csv', and created 'Cards in Boxes_cards.txt'
Removed 'list' column, renamed 'List10_cards.csv' to 'Trade Binder_cards.csv', and created 'Trade Binder_cards.txt'
Removed 'list' column, renamed 'List11_cards.csv' to 'Zedruu, the Greathearted_cards.csv', and created 'Zedruu, the Greathearted_cards.txt'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>